In [ ]:
#This notebook will sort the observations from the INT/WFC, and construct the lists needed
#by iraf for the reduction
#Each file will be renamed based on the type of observation, with the observed mjd added
#Assuming they are all downloaded in the same folder, the structure will be as follows:

#download_folder
#  |
#  + date 1 <yyyymmdd>
#  |    + bias
#  |    |   + bias_<mjd1>.fit
#  |    |   + bias_<mjd2>.fit
#  |    |   ....
#  |    |   + biaslist
#  |    |
#  |    + filter 1
#  |    |   + flats
#  |    |   |   + dome
#  |    |   |   |   + domeflat_<mjd1>.fit
#  |    |   |   |   + domeflat_<mjd2>.fit
#  |    |   |   |   ....
#  |    |   |   |   + flatlist
#  |    |   |   |
#  |    |   |   + sky
#  |    |   |       + skyflat_<mjd1>.fit
#  |    |   |       + skyflat_<mjd2>.fit
#  |    |   |       ....
#  |    |   |       + flatlist
#  |    |   |
#  |    |   +science
#  |    |       + object 1 <obj_name>_<mjd1>.fit
#  |    |       + object 2 <obj_name>_<mjd2>.fit
#  |    |       ....
#  |    |       + snlist
#  |    |
#  |    + filter 2
#  |    ....
#  |
#  + date 2 <yyyymmdd>
#  ....

#to properly do this, each file has to have its header read, from there the date, imtype,
#and name can be determined

In [1]:
#Import the needed functions
from astropy.io import fits
from pathlib import Path
import shutil

In [2]:
#First give the current location of the files and list them all
down_dir = Path('/Users/terwelj/Documents/VirtualMachine/Observing_ING/test_pipeline')

#The images in imlist will be sorted in their correct locations
imlist = sorted(down_dir.glob('*.fit'))

In [3]:
#A function that opens the file given, and returns date, imtype, name, band, assuming it is
#an INT/WFC image
def read_header(path):
    head = fits.open(path)[0].header
    #Observations before 10:00 belong to the previous observing night
    date_loc = head['DATE-OBS'].split('-')
    if int(head['UT'][:2])<10:
        date = date_loc[0]+date_loc[1]+str(int(date_loc[2])-1)
    else:
        date = date_loc[0]+date_loc[1]+date_loc[2]
    return {'mjd':head['MJD-OBS'], 'imtype':head['OBSTYPE'], 'name':head['OBJECT'],
            'band':head['WFFBAND'], 'bandsys':head['WFFPSYS'], 'date':date}

#A function that takes the data from the header, and decides the new location & name
def find_new_loc(base_path, head_info):
    #Select the date folder
    #(= date at start of night, e.g. obs on 5-1-23T03:00:00 are on night 4-1-23)
    loc = head_info['date']
    #Select bias / filter
    if head_info['imtype'] == 'BIAS':
        loc += '/bias'
        file_name = f'bias_{head_info["mjd"]}.fit'
        return base_path / loc, file_name
    else:
        loc += '/'+head_info['bandsys']+'_'+head_info['band']
    #Select flat/science
    if head_info['imtype'] == 'FLAT':
        loc += '/flats/dome'
        file_name = f'domeflat_{head_info["mjd"]}.fit'
    elif head_info['imtype'] == 'SKY':
        loc += '/flats/sky'
        file_name = f'skyflat_{head_info["mjd"]}.fit'
    elif head_info['imtype'] == 'TARGET':
        loc += '/science'
        file_name = f'{head_info["name"]}_{head_info["mjd"]}.fit'
    else:
        print('ERROR: imtype not recognised')
    return down_dir/loc, file_name

#Make a list of files to reduce in the target location
def make_list(loc, listname):
    imlist = loc.glob('*.fit')
    with open(loc/listname, 'w') as f:
        for im in imlist:
            f.write(im.name[:-4])
            f.write('\n')
    return

In [4]:
#Loop over all images and move them to the correct place
dlist = []
for im in imlist:
    head_info = read_header(im)
    target_loc, file_name = find_new_loc(down_dir, head_info)
    #Check if location exists, and if there already is a file with the same name
    target_loc.mkdir(parents=True, exist_ok=True)
    print(f'path {target_loc} exists now')
    file_list = sorted(target_loc.glob(file_name))
    if len(file_list) != 0:
        print('ERROR: file already exists, skipping this')
        print(f'loc: {target_loc} file: {file_name}')
        continue
    #Move the file
    shutil.move(down_dir/im, target_loc/file_name)
    #Keep a list of date_folders in which the iraf lists need to be made
    dlist.append(head_info['date'])

path /Users/terwelj/Documents/VirtualMachine/Observing_ING/test_pipeline/20221213/Sloan_g/science exists now


In [5]:
#Make the lists needed for iraf & check if stuff is missing
found_bias = False
for _ in set(dlist):
    for d in (down_dir/_).glob('*'):
        if d.name == 'bias':
            found_bias = True
            make_list(d, 'biaslist')
        elif d.is_dir():
            found_flats = False
            found_science = False
            try:
                make_list(d/'flats/dome', 'flatlist')
                found_flats = True
            except:
                pass
            try:
                make_list(d/'flats/sky', 'flatlist')
                found_flats = True
            except:
                pass
            if not found_flats:
                print(f'WARNING: No flat directory in {d}')
            try:
                make_list(d/'science', 'snlist')
                found_science = True
            except:
                pass
            if not found_science:
                print(f'WARNING: No science directory in {d}')
    if not found_bias:
        print(f'WARNING: No bias directory in {_}')